## Example deployment function

Using global data

# Orig version

In [1]:
import pandas as pd

In [2]:
def dissonance(
    resp_dict,
    model_paths,
    classifier,
):
    """_summary_

    Args:
        resp_dict (_type_): response dictionary with keys as questions and values as subjects' responses
        model_paths (_type_): paths to qnet models used for computing dissonances
        question_order (_type_): order of questions used by qnets specified in `model_paths`
        classifier (_type_, optional): pretrained classifier used to compute malingering probability. Defaults to None.

    Returns:
        _type_: dissonance(s) of subjects (and malingering prob, if classifier is provided)
    """
    import numpy as np
    from quasinet.qnet import load_qnet

    def _diss_linear(s, qnet, missing_value=0):
        diss = list()
        Ds = qnet.predict_distributions(s)

        for i in range(len(s)):
            if s[i] != "":
                if s[i] in Ds[i].keys():
                    diss.append(1 - Ds[i][s[i]] / np.max(list(Ds[i].values())))
                elif s[i] == "missing":
                    diss.append(missing_value)
                else:
                    diss.append(1)

        return np.array(diss)

    def _actual_sample_dissonance(
        data_sample, diss_models, diss_fcn, order, length, missing_value=0
    ):
        if order is None:
            order = range(length)

        sample = np.full(length, "", dtype="<U21")

        diss = [list() for model in diss_models]

        for i in order:
            if data_sample[i] == "":
                sample[i] = "missing"
            else:
                sample[i] = data_sample[i]
            for d, model in zip(diss, diss_models):
                d.append(diss_fcn(sample, model, missing_value))

        return sample, diss

    def _all_actual_samples_dissonance(
        data_samples, diss_models, diss_fcn, order, length, missing_value=0
    ):
        samples = list()
        dissonances = list()

        for data_sample in data_samples:
            samp, diss = _actual_sample_dissonance(
                data_sample, diss_models, diss_fcn, order, length, missing_value
            )
            samples.append(samp)
            dissonances.append(diss)

        return samples, dissonances

    def _dissonance_data_at_question(dissonances, questions_asked):
        return np.array(
            [np.hstack([d[questions_asked - 1] for d in diss]) for diss in dissonances]
        )

    diss_models = [
        load_qnet(
            model_path,
            gz=True,
        )
        for model_path in model_paths
    ]
    resp_df = pd.DataFrame(resp_dict).T

    resp_fmtd = (
        resp_df[diss_models[0].feature_names]
        .fillna(-9)
        .astype(int)
        .replace(-9, "")
        .to_numpy()
    )

    resp_diss = _all_actual_samples_dissonance(
        resp_fmtd,
        diss_models,
        _diss_linear,
        range(resp_fmtd.shape[1]),
        resp_fmtd.shape[1],
    )[1]

    resp_new = pd.DataFrame(_dissonance_data_at_question(resp_diss, resp_fmtd.shape[1]))

    proba = classifier.predict_proba(resp_new)

    output_dict = dict.fromkeys(
        resp_dict.keys(),
    )
    for key, val in zip(output_dict.keys(), proba[:, 1]):
        output_dict[key] = val

    return output_dict

In [3]:
resp_dict = pd.read_pickle("global-data-example-dict.pkl")
model_paths = [
    f"models/global/random_order_{model}_model_0.joblib.gz"
    for model in ["full"]  # , "neg", "pos"]
]
classifier = pd.read_pickle(f"classifiers/global/runif-{346}.pkl")

In [4]:
dissonance(resp_dict, model_paths, classifier)

{0: 0.01,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.02,
 6: 0.01,
 7: 0.0,
 8: 0.0,
 9: 0.04}

# Alt version

In [5]:
import pandas as pd
import numpy as np
from quasinet.qnet import load_qnet

VERBOSE = 0


def _diss_linear(s, qnet, missing_response="", missing_diss_value=0):
    Ds = qnet.predict_distributions(s)
    diss_values = []

    for i in range(len(s)):
        prob = float(Ds[i].get(str(s[i]), np.max(list(Ds[i].values()))))
        max_prob = max(Ds[i].values())
        diss_value = 1 - prob / max_prob if max_prob != 0 else 0
        if VERBOSE:
            print(i, "-->" + s[i] + "<", Ds[i], prob, max_prob, diss_value)
        diss_values.append(diss_value)

    if VERBOSE:
        print(diss_values)

    return diss_values


def dissonance(patients_responses, problem_type, model_path, classifier_path):
    """Computes dissonance values and malingering

    Args:
        patients_responses (_type_): dict of patients' responses
        problem_type (_type_): one of "global", "ptsd", "bond-court"
        model_path (_type_): path(s) to model(s) to use for computing dissonances ("full" for Qnet trained on all samples, "pos" for Qnet trained on positive samples, "neg" for Qnet trained on negative samples )
        classifier_path (_type_): path to pretrained classifier where positive class represents malingerers

    Returns:
        _type_: malingering risk (defined as class probability of the malingering class)
    """
    model = load_qnet(model_path)
    classifier = pd.read_pickle(classifier_path)

    all_data_samples = []

    for patient_response in patients_responses:
        for patient_id, responses in patient_response.items():
            resp_df = pd.DataFrame([responses], columns=model.feature_names)
            data_samples = (
                resp_df.fillna("")  # Replace missing values with empty strings
                .astype(str)  # Convert all values to strings
                .values
            )
            all_data_samples.append(data_samples[0])

    diss_values = [_diss_linear(sample, model) for sample in all_data_samples]
    proba = classifier.predict_proba(diss_values)

    # Create output dictionary
    output_dict = {}
    for idx, patient_response in enumerate(patients_responses):
        for patient_id in patient_response.keys():
            output_dict[patient_id] = proba[idx, 1]  # Assign the correct probability

    return output_dict


# Example usage
problem_type = "global"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/runif-{346}.pkl"


patients_responses = [
    {
        "0": {
            "20": "1",
            "31": 2.0,
            "36": 1.0,
            "51": 2.0,
            "60": 1.0,
            "75": 1.0,
            "85": 1.0,
            "115": 1.0,
            "123": 2.0,
            "147": 2.0,
            "150": 3.0,
            "179": 2.0,
            "312": 2.0,
            "355": 2.0,
            "436": 4.0,
            "572": 3.0,
            "719": 2.0,
            "737": 2.0,
            "747": 2.0,
            "817": 2.0,
            "855": 2.0,
            "938": 1.0,
            "983": 2.0,
            "1053": 2.0,
            "1060": 2.0,
            "1069": 1.0,
            "1132": 2.0,
            "4201": 1.0,
            "4203": 1,
            "4247": 1.0,
            "4250": 2.0,
            "4251": 2.0,
            "4254": 2.0,
            "4255": 1,
            "4262": 2.0,
            "4278": 1,
            "4280": 1,
            "4287": 1,
            "4289": 1,
            "4439": 1.0,
            "4441": 1.0,
            "4442": 2.0,
            "4452": 1.0,
            "4453": 0,
            "4454": 0,
            "4455": 0,
            "4456": 0.0,
            "4525": 1,
            "4527": 1,
            "4529": 3.0,
            "4531": 1.0,
            "4532": 2.0,
            "4545": 3.0,
            "4553": 1.0,
            "4555": 1,
            "4571": 1.0,
            "4574": 2.0,
            "4587": 1.0,
        }
    },
    {
        "9": {
            "20": 2.0,
            "31": 1.0,
            "36": 1.0,
            "46": 1.0,
            "51": 1.0,
            "53": 1.0,
            "61": 2.0,
            "62": 2.0,
            "76": 1.0,
            "85": 1.0,
            "109": 3.0,
            "110": 4.0,
            "111": 3.0,
            "123": 2.0,
            "149": 4.0,
            "210": 3.0,
            "255": 2.0,
            "576": 1.0,
            "603": 4.0,
            "840": 2.0,
            "841": 2.0,
            "855": 2.0,
            "881": 2.0,
            "927": 2.0,
            "928": 2.0,
            "936": 2.0,
            "1009": 3.0,
            "1041": 3.0,
            "1080": 3.0,
            "1125": 2.0,
            "4203": 1,
            "4247": 1.0,
            "4248": 4.0,
            "4250": 2.0,
            "4251": 3.0,
            "4255": 3,
            "4262": 1.0,
            "4278": 1,
            "4280": 1,
            "4287": 1,
            "4289": 1,
            "4355": 3.0,
            "4439": 3.0,
            "4441": 3.0,
            "4451": 2.0,
            "4452": 3.0,
            "4453": 0,
            "4454": 0,
            "4455": 0,
            "4456": 0.0,
            "4525": 2,
            "4527": 1,
            "4554": 2.0,
            "4555": 3,
            "4586": 1.0,
            "4587": 3.0,
            "4590": 1.0,
            "4596": 3.0,
        }
    },
    {"PHX1": {"20": "1", "31": 1.0}},
]

processed_responses = [
    {
        patient_id: {
            question_id: str(int(response))
            for question_id, response in patient_responses.items()
        }
        for patient_id, patient_responses in patient.items()
    }
    for patient in patients_responses
]


result = dissonance(processed_responses, problem_type, model_path, classifier_path)
print(result)

{'0': 0.01, '9': 0.04, 'PHX1': 0.0}


In [6]:
patients_responses = (
    pd.read_csv("../data/gibbons_global/gibbons_global.csv")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]


problem_type = "global"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/runif-{346}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.01,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.01,
 10: 0.02,
 11: 0.0,
 12: 0.01,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.04,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.01,
 24: 0.0,
 25: 0.02,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.01,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.02,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.01,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.03,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.01,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.01,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.

In [7]:
patients_responses = (
    pd.read_csv("../data/gibbons_global/gibbons_global.csv")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]


problem_type = "global"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/expert-{346}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.01,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.03,
 26: 0.0,
 27: 0.0,
 28: 0.03,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.02,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.01,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.0,
 100:

## PTSD

In [8]:
patients_responses = pd.read_csv("../data/ptsd/PTSD_cognet_test_processed.csv").to_dict(
    "index"
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "ptsd"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/runif-{211}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.11,
 1: 0.03,
 2: 0.16,
 3: 0.06,
 4: 0.1,
 5: 0.19,
 6: 0.13,
 7: 0.14,
 8: 0.03,
 9: 0.43,
 10: 0.07,
 11: 0.04,
 12: 0.16,
 13: 0.13,
 14: 0.13,
 15: 0.01,
 16: 0.22,
 17: 0.16,
 18: 0.29,
 19: 0.12,
 20: 0.29,
 21: 0.2,
 22: 0.17,
 23: 0.07,
 24: 0.04,
 25: 0.05,
 26: 0.05,
 27: 0.04,
 28: 0.02,
 29: 0.02,
 30: 0.04,
 31: 0.02,
 32: 0.19,
 33: 0.05,
 34: 0.15,
 35: 0.03,
 36: 0.29,
 37: 0.15,
 38: 0.08,
 39: 0.04,
 40: 0.03,
 41: 0.01,
 42: 0.08,
 43: 0.05,
 44: 0.11,
 45: 0.04,
 46: 0.32,
 47: 0.01,
 48: 0.12,
 49: 0.03,
 50: 0.13,
 51: 0.01,
 52: 0.28,
 53: 0.08,
 54: 0.0,
 55: 0.03,
 56: 0.05,
 57: 0.22,
 58: 0.13,
 59: 0.04,
 60: 0.3,
 61: 0.15,
 62: 0.02,
 63: 0.23,
 64: 0.03,
 65: 0.03,
 66: 0.15,
 67: 0.04,
 68: 0.01,
 69: 0.08,
 70: 0.1,
 71: 0.14,
 72: 0.1,
 73: 0.03,
 74: 0.07,
 75: 0.08,
 76: 0.05,
 77: 0.05,
 78: 0.03,
 79: 0.13,
 80: 0.04,
 81: 0.0,
 82: 0.06,
 83: 0.1,
 84: 0.05,
 85: 0.1,
 86: 0.37,
 87: 0.0,
 88: 0.09,
 89: 0.04,
 90: 0.22,
 91: 0.03,
 92: 0.1

In [9]:
patients_responses = pd.read_csv("../data/ptsd/PTSD_cognet_test_processed.csv").to_dict(
    "index"
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "ptsd"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/expert-{211}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.1,
 1: 0.06,
 2: 0.17,
 3: 0.11,
 4: 0.05,
 5: 0.2,
 6: 0.05,
 7: 0.05,
 8: 0.16,
 9: 0.45,
 10: 0.1,
 11: 0.05,
 12: 0.15,
 13: 0.07,
 14: 0.19,
 15: 0.0,
 16: 0.11,
 17: 0.28,
 18: 0.42,
 19: 0.04,
 20: 0.43,
 21: 0.32,
 22: 0.21,
 23: 0.25,
 24: 0.07,
 25: 0.06,
 26: 0.21,
 27: 0.07,
 28: 0.02,
 29: 0.13,
 30: 0.0,
 31: 0.09,
 32: 0.31,
 33: 0.09,
 34: 0.12,
 35: 0.0,
 36: 0.28,
 37: 0.18,
 38: 0.16,
 39: 0.28,
 40: 0.05,
 41: 0.02,
 42: 0.16,
 43: 0.05,
 44: 0.09,
 45: 0.04,
 46: 0.33,
 47: 0.02,
 48: 0.13,
 49: 0.0,
 50: 0.27,
 51: 0.05,
 52: 0.31,
 53: 0.04,
 54: 0.07,
 55: 0.12,
 56: 0.07,
 57: 0.37,
 58: 0.44,
 59: 0.03,
 60: 0.32,
 61: 0.03,
 62: 0.11,
 63: 0.27,
 64: 0.12,
 65: 0.02,
 66: 0.05,
 67: 0.03,
 68: 0.01,
 69: 0.14,
 70: 0.0,
 71: 0.25,
 72: 0.06,
 73: 0.12,
 74: 0.08,
 75: 0.14,
 76: 0.07,
 77: 0.01,
 78: 0.08,
 79: 0.09,
 80: 0.06,
 81: 0.08,
 82: 0.1,
 83: 0.12,
 84: 0.01,
 85: 0.12,
 86: 0.36,
 87: 0.01,
 88: 0.18,
 89: 0.05,
 90: 0.29,
 91: 0.04,
 92: 0.

## Bond court

In [10]:
patients_responses = (
    pd.read_csv("../data/bondcourt/CCHHS_SUD_details.csv")
    .drop("Interview_ID", axis="columns")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "bond-court"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/runif-{42}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.01,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.02,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.01,
 24: 0.0,
 25: 0.01,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.01,
 62: 0.0,
 63: 0.0,
 64: 0.01,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.1,
 85: 0.0,
 86: 0.0,
 87: 0.01,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.01,
 96: 0.0,
 97: 0.0,
 98: 0.0,
 99: 0.0,
 1

In [11]:
patients_responses = (
    pd.read_csv("../data/bondcourt/CCHHS_SUD_details.csv")
    .drop("Interview_ID", axis="columns")
    .fillna(-9)
    .astype(int)
    .replace(-9, "")
    .to_dict("index")
)
processed_responses = [
    {patient: patients_responses[patient]} for patient in patients_responses
]
problem_type = "bond-court"
model_path = f"models/{problem_type}/random_order_full_model_0.joblib.gz"
classifier_path = f"classifiers/{problem_type}/expert-{42}.pkl"
dissonance(processed_responses, problem_type, model_path, classifier_path)

{0: 0.0,
 1: 0.0,
 2: 0.04,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.03,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.07,
 16: 0.0,
 17: 0.05,
 18: 0.01,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.01,
 23: 0.03,
 24: 0.0,
 25: 0.06,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.02,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.01,
 45: 0.0,
 46: 0.0,
 47: 0.01,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.01,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.1,
 65: 0.0,
 66: 0.0,
 67: 0.03,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.02,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.06,
 79: 0.01,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.03,
 85: 0.01,
 86: 0.0,
 87: 0.04,
 88: 0.0,
 89: 0.0,
 90: 0.05,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.01,
 96: 0.0,
 97: 0.0,
 98: 0.0,